# Preprocess HyperNeRF data to NSFF data style

In [5]:
import json
import numpy as np
import imageio
import pdb
from pathlib import Path
import cv2
import os
import shutil
from PIL import Image

# @markdown The base directory for all captures. This can be anything if you're running this notebook on your own Jupyter runtime.
save_dir = './data/'  
capture_name = 'kubric_single_car_rand_v2' 
# The root directory for this capture.
root_dir = Path(save_dir, capture_name)

rgb_raw_dir = root_dir / 'images'


rgb_list = list(sorted(rgb_raw_dir.glob( '*.png')))
image_size = imageio.imread(rgb_list[0]).shape[:2]


# @title Save dataset split to `dataset_nsff.json`.

with open(root_dir / 'dataset.json','r') as f:
    dataset_info = json.load(f)

all_ids = list(map(lambda x: x.stem, rgb_list))
val_ids = dataset_info['val_ids']
test_ids = [id for id in all_ids if 'test' in id]
train_ids = sorted(set(all_ids) - set(val_ids) - set(test_ids))

print(f'Num of train: {len(train_ids)}')
print(f'Num of val: {len(val_ids)}')

dataset_json = {
    'count': len(all_ids),
    'num_exemplars': len(train_ids),
    'ids': all_ids,
    'train_ids': train_ids,
    'val_ids': val_ids,
    'test_ids': test_ids
}

dataset_json_path = root_dir / 'dataset_nsff.json'
with dataset_json_path.open('w') as f:
    json.dump(dataset_json, f, indent=2)



Num of train: 200
Num of val: 0


In [4]:
test_ids

['test_rgba_00000',
 'test_rgba_00001',
 'test_rgba_00002',
 'test_rgba_00003',
 'test_rgba_00004',
 'test_rgba_00005',
 'test_rgba_00006',
 'test_rgba_00007',
 'test_rgba_00008',
 'test_rgba_00009',
 'test_rgba_00010',
 'test_rgba_00011',
 'test_rgba_00012',
 'test_rgba_00013',
 'test_rgba_00014',
 'test_rgba_00015',
 'test_rgba_00016',
 'test_rgba_00017',
 'test_rgba_00018',
 'test_rgba_00019',
 'test_rgba_00020',
 'test_rgba_00021',
 'test_rgba_00022',
 'test_rgba_00023',
 'test_rgba_00024',
 'test_rgba_00025',
 'test_rgba_00026',
 'test_rgba_00027',
 'test_rgba_00028',
 'test_rgba_00029',
 'test_rgba_00030',
 'test_rgba_00031',
 'test_rgba_00032',
 'test_rgba_00033',
 'test_rgba_00034',
 'test_rgba_00035',
 'test_rgba_00036',
 'test_rgba_00037',
 'test_rgba_00038',
 'test_rgba_00039',
 'test_rgba_00040',
 'test_rgba_00041',
 'test_rgba_00042',
 'test_rgba_00043',
 'test_rgba_00044',
 'test_rgba_00045',
 'test_rgba_00046',
 'test_rgba_00047',
 'test_rgba_00048',
 'test_rgba_00049',


In [3]:
# read scene info
if (root_dir / 'scene_gt.json').exists():
    with open(root_dir / 'scene_gt.json','r') as f:
        scene_info = json.load(f)
else:
    with open(root_dir / 'scene.json','r') as f:
        scene_info = json.load(f)

near = scene_info['near'] / scene_info['scale']
far = scene_info['far'] / scene_info['scale']

# read camera info

c2w_mats = []
hwfs = []
bounds_mats = []
for i, rgb in enumerate(rgb_list):
    with open(camera_dir / f'{rgb.stem}.json','r') as f:
        camera_info = json.load(f)
    hwf = np.concatenate([camera_info['image_size'], [camera_info['focal_length']]])
    c2w_rot = np.array(camera_info['orientation']).T
    c2w_trans = camera_info['position']

    c2w_mat = np.zeros([4,4])
    c2w_mat[:3,:3] = c2w_rot
    c2w_mat[3,:3] = c2w_trans
    c2w_mat[3,3] = 1.
    c2w_mats.append(c2w_mat)
    hwfs.append(hwf[..., None])
    bounds_mats.append(np.array([near, far]))

c2w_mats = np.array(c2w_mats)
hwfs = np.array(hwfs).transpose([1,2,0])

poses = c2w_mats[:, :3, :4].transpose([1,2,0])
poses = np.concatenate([poses, hwfs], 1)

# must switch to [-y, x, z] from [x, -y, -z], NOT [r, u, -t]
poses = np.concatenate([poses[:, 1:2, :], poses[:, 0:1, :], 
                        -poses[:, 2:3, :], 
                        poses[:, 3:4, :], 
                        poses[:, 4:5, :]], 1)

save_arr = []
for i in range((poses.shape[2])):
    save_arr.append(np.concatenate([poses[..., i].ravel(), bounds_mats[i]], 0))

save_arr = np.array(save_arr)
np.save(root_dir / 'poses_bounds.npy', save_arr)

with open( root_dir / 'scene.json', 'w') as f:
    json.dump(scene_info, f, indent=2)

In [2]:
# prepare for freeze test
freeze_rgb_list = sorted((freeze_test_dir / 'rgb' / image_scale).glob( '*.png'))
if freeze_test_dir.exists():
    target_image_dir = freeze_test_dir / 'images'
    target_image_dir.mkdir(exist_ok=True)
    for rgb in list(freeze_rgb_list):
        shutil.copy(str(rgb), str(target_image_dir))


    c2w_mats = []
    hwfs = []
    bounds_mats = []
    for i, rgb in enumerate(freeze_rgb_list):
        with open(cam_freeze_test_dir / f'{rgb.stem}.json','r') as f:
            camera_info = json.load(f)
        hwf = np.concatenate([camera_info['image_size'], [camera_info['focal_length']]])
        c2w_rot = np.array(camera_info['orientation']).T
        c2w_trans = camera_info['position']

        c2w_mat = np.zeros([4,4])
        c2w_mat[:3,:3] = c2w_rot
        c2w_mat[3,:3] = c2w_trans
        c2w_mat[3,3] = 1.
        c2w_mats.append(c2w_mat)
        hwfs.append(hwf[..., None])
        bounds_mats.append(np.array([near, far]))

    c2w_mats = np.array(c2w_mats)
    hwfs = np.array(hwfs).transpose([1,2,0])

    poses = c2w_mats[:, :3, :4].transpose([1,2,0])
    poses = np.concatenate([poses, hwfs], 1)

    # must switch to [-y, x, z] from [x, -y, -z], NOT [r, u, -t]
    poses = np.concatenate([poses[:, 1:2, :], poses[:, 0:1, :], 
                            -poses[:, 2:3, :], 
                            poses[:, 3:4, :], 
                            poses[:, 4:5, :]], 1)

    save_arr = []
    for i in range((poses.shape[2])):
        save_arr.append(np.concatenate([poses[..., i].ravel(), bounds_mats[i]], 0))

    save_arr = np.array(save_arr)
    np.save(freeze_test_dir / 'poses_bounds.npy', save_arr)

